### Performance

On average, SVG circuit drawer is 8 times faster than MPL circuit drawer. The following graphs show how time scales with number of qubits and circuit depth for both drawers:


![Number of Qubits](https://github.com/EgrettaThula/svg_circuit_drawer/blob/main/_static/num-qubits.png?raw=true)

![Depth](https://github.com/EgrettaThula/svg_circuit_drawer/blob/main/_static/depth.png?raw=true)

Let's create a large circuit and draw it using both SVG and MPL circuit drawers.

In [1]:
from qiskit.circuit.random import random_circuit
from svg_drawer.circuit_visualization import circuit_drawer

large_circuit = random_circuit(20, 150, 2, seed = 1)

In [ ]:
circuit_drawer(large_circuit)

In [ ]:
large_circuit.draw('mpl')

### Better Styling

SVG supports CSS; a rich easy to use standard styling language.

Lets create a circuit and use it to demonstrate the richness of CSS-based styling:

In [ ]:
from qiskit.quantum_info import random_unitary
op2 = random_unitary(4)
op3 = random_unitary(8)

circ2 = random_circuit(5, 5, 2, seed = 2)
circ2.unitary(op2, [1, 2], label = '   Test   ')
circ2.measure_all()

circuit_drawer(circ2)

In [ ]:
style = {
    # Reduce the radius of circuits in controlled gates:
    'circle.control': { 'r': 3 },
    # Make the gray area around the barrier less opaque: 
    'rect.barrier': { 'opacity': 0.15 },
    # Display unitary bits in white color using a smaller font:
    'text.unitary.bit': { 'fill':'white', 'font-size': '0.8em' },
    # Display gate angles in yellow: 
    'text.angle': { 'fill':'#ffffaa' },
    # Use light gray for classical wires:
    'line.cwire': { 'stroke': 'lightgray' },
    # Display classical wires bundle symbol and number of bits in green:
    'line.cwire.bundle': { 'stroke': 'green' },
    'text.cwire.size': { 'fill':'green', 'font-weight': 'bold' },
    # Display colorful measurement instruments:
    'rect.measure': { 'fill': 'indigo' },
    'line.measure-symbol': { 'stroke': 'yellow' },
    'path.measure-symbol': { 'stroke': 'orange' },
}

circuit_drawer(circ2, style = style)

### Advanced Styles:

Now, let's try something more advanced. In this [video](https://www.youtube.com/watch?v=Ejx61b2cIG4) Aram Harrow uses a nice circuit plots with the following properties:

-  Comic fonts.
-  Gradient colors.
-  Rounded corners.
-  Moreover, he chose for time to flow from right to left!

In [ ]:
harrow_defs = """
    <linearGradient id="harrow" x2="0%" y2="100%">
        <stop offset="5%" stop-color="#598fe8" />
        <stop offset="95%" stop-color="#192766" />
    </linearGradient>
    """

harrow_style = {
    '': { 'background-color': '#2d3732' },
    'rect:not(.barrier):not(.measure)': { 'fill': 'url(#harrow)', 'rx': 5, 'stroke': '#598fe8', 'stroke-width': 1 },
    'rect.measure': { 'stroke': '#ffffff', 'stroke-width': 2 },
    'line:not(.cwire):not(.measure):not(.measure-symbol)': { 'stroke': '#598fe8' },
    'circle.control': { 'fill':'#192766' },
    'text:not(.angle)': { 'fill': '#ffffff' },
    'text.unitary.bit': { 'display': 'none' },
    
    'text.unitary': { 'font-family': 'Comic Sans MS', 'font-size': '1.2em' },
    
    'text.qwire.bit': { 'font-size': 0 },
    'text.qwire tspan.initial-state': { 'font-family': 'Comic Sans MS', 'font-size': '12pt' },
    
    '>g': { 'transform-box': 'fill-box', 'transform-origin': 'center', 'transform': 'scale(-1, 1)' },
    'text': { 'transform-box': 'fill-box', 'transform-origin': 'center', 'transform': 'scale(-1, 1)', 'white-space': 'pre' },
}

from qiskit.circuit import Gate
from qiskit.circuit.library import QuantumVolume
circ3 = QuantumVolume(4, 5, seed = 5).decompose()

for index, instruction in enumerate(circ3.data):
    instruction[0].label = '   $U_{{{x}}}$'.format(x = index + 1)

circ3.measure_all()

circuit_drawer(circ3, initial_state = True, plot_barriers = False, defs = [harrow_defs], style = harrow_style)

### Rich Snips

"Rich snips" is a unique feature for SVG drawer which has no equivalent in MPL drawer. Rich snip is a piece of information related to the circuit or part of it, which is displayed when user clicks on some active part in the circuit plot. This feature depends on SVG support for animations.

Currently, two types of rich snips are supported:
-  Calibrations.
-  Decomposition.

More rich snip types are to come.

In [ ]:
from qiskit import pulse
from qiskit.pulse.library import Gaussian, Constant
from qiskit.test.mock import FakeValencia

backend = FakeValencia()

with pulse.build(backend, name='hadamard') as h_sch:
    pulse.play(Gaussian(duration = 128, amp = 0.1, sigma=16), pulse.drive_channel(0))

with pulse.build(backend) as x_sch:
    pulse.play(Constant(duration = 10, amp = 0.1), pulse.drive_channel(0))
"""
circ4 = random_circuit(5, 5, max_operands = 2, seed = 1)
circ4.h(0)
circ4.x(1)
circ4.measure_all()
"""

circ2.add_calibration('h', [0], h_sch)
circ2.add_calibration('x', [1], x_sch)

circuit_drawer(circ2, rich_snips = True)

### And More

- **Smaller file sizes**: on average Saving the MPL circuit plot as a .svg file results in a three times larger files.
- **No 2^16 pixels limit**: there is a hard-coded limit of [2^16 pixels](https://github.com/matplotlib/matplotlib/blob/main/src/_backend_agg_wrapper.cpp) on the image size that can be generated by matplotlib.
For example, try to draw the following circuit using both drawers:

In [7]:
huge_circuit = random_circuit(20, 500, 2, seed = 1)